# Convolutional Neural Network (CNN)

### Kütüphanelerin İçeri Aktarılması

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

* Veri artırımı yapmak için <b>`ImageDataGenerator`</b> kullanıldı.

In [2]:
tf.__version__

'2.16.1'

## Part 1 - Veri Ön İşleme

### Eğitim Seti Ön İşleme

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.


* <b> `rescale`</b>, piksel değerleri 0-1 arasına ölçeklendi.
  
* <b>`shear_range`</b>, Bir görüntüyü belirli bir açıda yatay veya dikey olarak kaydırarak deformasyona uğratır.
   * Genellikle küçük bir <b>`shear_range`</b> değeri kullanılır. Çünkü büyük değerler görüntüleri bozabilir ve öğrenme süreci olumsuz etkilenebilir.

* <b>`zoom_range`</b>, görüntüyü rastgele yakınlaştırır.

* <b>`horizontal_flip`</b>, görüntüyü yatay olarak rastgele çevirir.

<hr>

* <b>`target_size`</b>, dataset'ten yüklenen görüntülerin boyutunu belirtir.
  
* <b>`batch_size`</b>, eğitim sırasında kullanılacak mini-batch boyutunu belirtir.

* <b>`class_mode`</b>, iki sınıflı bir sınıflandırma problemi olduğu için <b>`binary`</b> kullanıldı.

### Test Seti Ön İşleme

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


* <b>`test_datagen = ImageDataGenerator(rescale=1./255)`</b> ile piksel değerleri 0-1 arasına ölçeklenir.

## Part 2 - CNN İnşa Etme

### CNN Başlatma

In [5]:
cnn = tf.keras.models.Sequential()

### Adım 1 - Convolution Layer

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

C:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


* <b>`filters`</b> katmandaki filtre sayısını belirtir. (Daha fazla filtre hesaplama maliyetini artırabilir.) <br>

* <b>`kernel_size`</b> Future map için boyut belirtir. (3x3 boyutunda bir future map olacağı belirtilmiştir.)
  
* <b>`input_shape`</b>64x64 boyutunda renkli bir görüntü verileceği belirtilmiş. [64, 64, 3]
  * <b>`input_shape=[64, 64, 1]`</b> olsaydı 64x64 boyutunda siyah beyaz bir görüntü verileceği belirtilirdi.

### Adım 2 - Pooling Layer

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

* <b>`pool_size=2`</b>, MaxPooling işlemi sırasında kullanılacak future map boyutu belirtilir.
  
* <b>`strides=2`</b>, Future map'in her adımda ne kadar kaydırılacağını belirtir.
  * Burada her adımda 2 piksel sağa, 2 piksel aşağıya kaydırılması gerektiği belirtilmiş.

### İkinci bir Convolution Layer Eklenmesi

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

* Bu katmanın eklenmesinin nedenleri:
   * Daha karmaşık özelliklerin öğrenilmesi
   * Modelin temsil gücünün artırılması
   * Overfitting'in azaltılması

### Adım 3 - Flattening Layer

In [9]:
cnn.add(tf.keras.layers.Flatten())

* Flattening katmanı sayesinde Full Connection Katmanı'na giriş olarak kullanılmak üzere veriyi hazırlar.

### Adım 4 - Full Connection Layer

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

* <b>`units`</b> parametresi ile gizli katmandaki nöron sayısı belirtilmiştir.
   * Daha fazla nöron, modelin daha karmaşık ilişkileri öğrenmesine olanak tanır.
   * Aynı zamanda daha fazla hesaplama maliyetine ve overfitting'e neden olabilir.

### Adım 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

* Sadece 2 sınıflı sınıfladırma yapıldığı için <b>`sigmoid`</b> kullanıldı.
  
* Çok sınıflı sınıflandırma yapılsaydı activation için <b>`softmax`</b> seçilirdi.

## Part 3 - CNN'nin Eğitilmesi

### CNN'nin Derlenmesi

In [12]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

* Modelin derlenmesi için <b>`compile`</b> parametresi kullanılır.
  
* Modelin kayıp fonksiyonu için <b>`binary_crossentropy`</b> kullanılacağı belirtilir.
  
* Modelin performansını değerlendirmek için <b>`accuracy`</b> doğruluk metriğinin kullanılacağı belirtilir.

### CNN'in Eğitim seti üzerinde eğitilmesi ve Test seti üzerinde değerlendirilmesi


In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs=25)

Epoch 1/25


C:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 53s 205ms/step - accuracy: 0.5290 - loss: 0.7015 - val_accuracy: 0.6115 - val_loss: 0.6468
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.6331 - loss: 0.6406 - val_accuracy: 0.7060 - val_loss: 0.5750
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - accuracy: 0.6893 - loss: 0.5812 - val_accuracy: 0.6980 - val_loss: 0.5682
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 72ms/step - accuracy: 0.7326 - loss: 0.5376 - val_accuracy: 0.7630 - val_loss: 0.5125
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.7450 - loss: 0.5166 - val_accuracy: 0.7800 - val_loss: 0.4742
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 72ms/step - accuracy: 0.7611 - loss: 0.4987 - val_accuracy: 0.7720 - val_loss: 0.4942
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 72ms/step - accuracy: 0.7750 - loss: 0.4684 - val_accuracy: 0.7885 - val_loss: 0.4699
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.7821 - loss: 0.4570 - val_accura

* <b>`x=training_set`</b> eğitim veri setini belirtir.
  
* <b>`validation_data=test_set`</b> doğrulama veri setini belirtir.

* <b>`batch_size=32`</b> kullanıldığı ve eğitim seti 8000 görüntüden oluştuğu için her bir epoch tamamlanması için 250 iterasyon gerekmekte. [8000 / 32 = 250]

* <b>`accuracy`</b> metriğine göre %90 oranında doğruluk oranına ulaşmış bulunmaktayız.

## Part 4 - Tahmin

In [20]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
sonuc = cnn.predict(test_image/255.0)

if sonuc[0][0] > 0.5:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [21]:
print(prediction)

dog
